In [8]:
import cupy as cp

def get_nonzero_ragged_and_indices_structured(x):
    K, num_qubits, _, _ = x.shape
    x_flat = x.reshape(K, num_qubits, -1)  # shape (K, num_qubits, 9)

    nonzero_values = []
    nonzero_indices_plus1 = []

    for k in range(K):
        values_row = []
        indices_row = []
        for q in range(num_qubits):
            row = x_flat[k, q]
            mask = row != 0
            values = row[mask]
            indices = cp.nonzero(mask)[0] + 1  # 1-based indices
            values_row.append(values)
            indices_row.append(indices)
        nonzero_values.append(values_row)
        nonzero_indices_plus1.append(indices_row)

    return nonzero_values, nonzero_indices_plus1

x = cp.array([
    [[[1,  2,  0], [1,  0, 0], [0,  2, 0]],
     [[-1, 0,  3], [0, 0, 1], [0, 1, 0]]]
])  # shape (1, 2, 3, 3)

values, indices = get_nonzero_ragged_and_indices_structured(x)

print(values)
print(indices)


[[array([1, 2, 1, 2]), array([-1,  3,  1,  1])]]
[[array([1, 2, 4, 8], dtype=int64), array([1, 3, 6, 8], dtype=int64)]]
